# Introduction to Spatial Data

Today we will introduce the basics of working with spatial data, including loading spatial datasets as shapefiles or CSV files, projecting data, performing geometric operations, spatially joining multiple datasets together, and simple mapping.

In [ ]:
!pip install geopandas mapclassify pandas shapely

In [ ]:
import geopandas as gpd
import pandas as pd

## 1. Quick overview of key concepts

### What is GIS?

GIS stands for geographic information system. GIS software lets you work with spatial data, that is, data associated with locations on the Earth. These locations are represented with coordinates: longitude (x), latitude (y), and often elevation (z). With GIS software you can collect, edit, query, analyze, and visualize spatial data. Examples of GIS software include ArcGIS, QGIS, PostGIS, and GeoPandas.

### Some terminology:

- **geoid**: (that's *gee-oid*) the surface of the earth's gravity field, which approximates mean sea level
- **spheroid** or **ellipsoid** (interchangeable terms): a model that smoothly approximates the geoid
- **datum**: based on spheroid but incorporates local variations in the shape of the Earth. Used to describe a point on the Earth's surface, such as in latitude and longitude.
  - WGS84 (World Geodetic Survey 1984 datum) uses the WGS84 spheroid
  - The latitude and longitude coordinates of some point differ slightly based on the datum. GPS uses WGS84.
- **coordinate reference system** (CRS) or spatial reference system (SRS): a series of parameters that [define](http://spatialreference.org/) the coordinate system and spatial extent (aka, domain) of some dataset.
- **geographic coordinate system** (GCS): specifies a datum, spheroid, units of measure (such as meters), and a prime meridian
- **projected coordinate system** or map projection: projects a map of the Earth's 3-D spherical surface onto a flat surface that can be measured in units like meters. Here's a [list of projections](https://en.wikipedia.org/wiki/List_of_map_projections).
- **eastings** and **northings**: the x and y coordinates of a projected map, usually measured in meters
- **false origin**: the 0,0 origin point from which eastings and northings are measured on a map, usually the lower left corner rather than the center
- **PROJ.4**: a library to convert/project spatial data with consistent CRS [parameter names](https://github.com/OSGeo/proj.4/wiki/GenParms)

### Common CRS parameters (and their PROJ.4 names):

- datum (datum)
- ellipse (ellps)
- projection (proj)
  - the name of the projected coordinate system, such as Albers Equal Area (aea) or Lambert Conformal Conic (lcc)
- standard parallels (lat_1, lat_2)
  - where the projection surface touches the globe - at the standard parallels, the projection shows no distortion
- central meridian and latitude of origin (lon_0, lat_0)
  - the origin of the projection's x and y coordinates (eastings and northings) - usually the center of the map projection
- false easting and false northing (x_0, y_0)
  - offsets to add to all your eastings and northings - usually used to make all the coordinates on the map positive numbers by starting 0,0 at the lower left corner rather than the center of the map (see false origin, above)

### Common projection types:

- *equal area* projections: maintain area at the expense of shape, distance, and direction - such as the [Albers Equal Area](https://en.wikipedia.org/wiki/Albers_projection) projection
- *conformal* projections: maintain shapes at the expense of area, distance, and direction - such as the [Lambert Conformal Conic](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) projection
- *equidistant* projections: [preserve distance](https://en.wikipedia.org/wiki/Map_projection#Equidistant) from one point or along all meridians and parallels
- *azimuthal* projections: maintain direction from one point to all other points - such as an [orthographic](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) projection
- others compromise to minimize overall distortion or aim for aesthetic value - such as the [Robinson](https://en.wikipedia.org/wiki/Robinson_projection) projection

## 2. Loading spatial data

You can use a GIS like ArcGIS or QGIS to open a spatial data file (typically a shapefile, GeoJSON file, or CSV file with lat-long columns). Today we'll introduce the basic concepts of spatial data and GIS operations using [geopandas](http://geopandas.org/user.html), which spatializes pandas dataframes. It uses the [shapely](https://shapely.readthedocs.io/en/latest/manual.html) package for geometry. We'll focus on common, shared concepts and operations, rather than "how-to" in the user interface of a specific GIS program.

### 2a. Loading a shapefile

Where to get census shapefiles? https://www.census.gov/cgi-bin/geo/shapefiles/index.php

The term "shapefile" is a misnomer... a shapefile is actually a folder containing multiple files that contain spatial geometry, attribute data, projection information, etc: https://en.wikipedia.org/wiki/Shapefile

In [ ]:
# tell geopandas to read a shapefile with its read_file() function, passing in the shapefile folder
# this produces a GeoDataFrame
gdf = gpd.read_file(
    "https://raw.githubusercontent.com/gboeing/ppd534/main/data/tl_2017_06_tract/tl_2017_06_tract.shp"
)
gdf.shape

In [ ]:
# just like regular pandas, see the first 5 rows of the GeoDataFrame
# this is a shapefile of polygon geometries, that is, tract boundaries
gdf.head()

In [ ]:
# mapping is as easy as calling the GeoDataFrame's plot method
ax = gdf.plot()

In [ ]:
# just like in regular pandas, we can filter and subset the GeoDataFrame
# retain only tracts in LA, OC, Ventura counties
counties = ["037", "059", "111"]
gdf_tracts = gdf[gdf["COUNTYFP"].isin(counties)]
gdf_tracts.shape

In [ ]:
# what is the CRS?
# this derives from the shapefile's .prj file
# always make sure the shapefile you load has prj info so you get a CRS attribute!
gdf_tracts.crs

### 2b. Loading a CSV file

Often, you won't have a shapefile (which is explicitly spatial), but rather a CSV file which is implicitly spatial (contains lat-lng columns). If you're loading a CSV file (or other non-explicitly spatial file type) of lat-lng data:

  1. first load the CSV file as a DataFrame the usual way with pandas
  2. then create a new geopandas GeoDataFrame from your DataFrame
  3. manually create a geometry column
  4. set the CRS

In [ ]:
# load rental listings data as a regular pandas dataframe
df = pd.read_csv("https://raw.githubusercontent.com/gboeing/ppd534/main/data/listings-la_oc_vc.csv")
df.shape

In [ ]:
# examine first five rows
df.head()

**Always define the CRS** if you are manually creating a GeoDataFrame! Earlier, when we loaded the shapefile, geopandas loaded the CRS from the shapefile itself. But our CSV file is not explicitly spatial and it contains no CRS data, so we have to tell it what it is. In our case, the CRS is EPSG:4326, which is WGS84 lat-lng data, such as for GPS. Your data source should always tell you what CRS their coordinates are in. If they don't, ask! Don't just guess.

In [ ]:
# create point geometries from the lng and lat coordinates
# notice the shapely points are represented as lng, lat so that they are equivalent to x, y
geometry = gpd.points_from_xy(x=df["longitude"], y=df["latitude"])

# create a new geopandas geodataframe manually from the pandas dataframe
# specify the geometries and the crs
gdf_listings = gpd.GeoDataFrame(df, geometry=geometry, crs="epsg:4326")
gdf_listings.shape

In [ ]:
gdf_listings.head()

In [ ]:
# what's the CRS
gdf_listings.crs

## 3. Projection, part I

Your datasets need to be in the same CRS if you want to work with them together. If they're not, then project one or both of them so they're in the same CRS.

In [ ]:
gdf_tracts.crs == gdf_listings.crs

In [ ]:
# project the tracts geodataframe to the CRS of the listings geodataframe
gdf_tracts = gdf_tracts.to_crs(gdf_listings.crs)
gdf_tracts.crs == gdf_listings.crs

**Be careful**: heed the difference between `gdf.crs` and `gdf.to_crs()`. The first tells you the geodataframe's current CRS. The latter projects the geodataframe to a new CRS.

## 4. Spatial predicates and operations

GIS and spatial analysis use common "computational geometry" predicates and operations like intersects, within, dissolve, and buffer.

  - *intersects* tells you if each geometry in one dataset intersects with some other (single) geometry
  - *within* tells you if each geometry in one dataset is wholly contained within some other (single) geometry
  - *dissolve* lets you aggregate data (merge their geometries together) if they share some attribute in common: this is the spatial equivalent of pandas's groupby function
  - *buffer* identifies a zone some distance around a geometry for proximity analysis
  
A spatial predicate tests the relationship between geometric objects and returns Trues or Falses (see the first two bullet points above, as well as touches, crosses, contains, overlaps, etc). A spatial operation takes geometric objects as inputs, and outputs different geometric objects (see the final two bullet points above, as well as union, intersection, difference, etc). Many predicates and operations exist in the world of GIS, but these are among the most common and useful.

### 4a. intersects

Example: I want to find all the tracts that have at least 1 rental listing within their boundaries. So, I'm going to intersect the tracts with a single geometry object that represents all the listings.

In [ ]:
# create a single, unified MultiPoint object containing all the listings' geometries
# use geopandas union_all method to get a single geometry object representing all the points
unified_listings = gdf_listings["geometry"].union_all()
type(unified_listings)

In [ ]:
unified_listings

In [ ]:
# get the tracts that spatially-intersect with anything in the listings dataset
mask = gdf_tracts.intersects(unified_listings)
gdf_tracts[mask].shape

In [ ]:
# how many tracts didn't intersect any rental listings?
gdf_tracts[~mask].shape

### 4b. dissolve

Example: I want to merge all the tracts in each county to aggregate them up to the county level. This will merge all tract-level geometries into new county-level geometries.

In [ ]:
# dissolve lets you aggregate data based on shared values in some column, such as county fips codes
gdf_tracts.head()

In [ ]:
# dissolve the tracts into counties
# aggregate their numerical columns by summing them
agg = {"ALAND": "sum", "AWATER": "sum"}
gdf_counties = gdf_tracts.dissolve("COUNTYFP", aggfunc=agg)
gdf_counties

In [ ]:
# quick and dirty map of our 3 counties
ax = gdf_counties.plot(cmap="plasma")

### 4c. within

Example: I want to find all the rental listings in Orange County. But my rental listings don't contain any explicit tract or county information: they only tell me lat-long. But I can use those lat-long coordinates to find which listings fall *within* the geometry (spatial boundary) of Orange County.

In [ ]:
# get orange county's geometry
oc_geometry = gdf_counties.loc["059", "geometry"]
type(oc_geometry)

In [ ]:
oc_geometry

In [ ]:
# find all the listings within orange county
mask = gdf_listings.within(oc_geometry)
oc_listings = gdf_listings[mask]
oc_listings.shape

## 5. Projection, part II

In [ ]:
# map the OC rental listings
ax = oc_listings.plot()

In [ ]:
oc_listings.head()

In [ ]:
# you can easily calculate buffers
# buffer creates a polygon around your geometry with some specified distance
oc_listings_buffered = oc_listings.buffer(distance=0.1)  # what are these units? 0.1 what?
oc_listings_buffered.head()

In [ ]:
ax = oc_listings_buffered.plot(edgecolor="k", alpha=0.3)

But these buffers are weird because the data are not projected. They're all in lat-long degrees. Let's project it to a **projected coordinate system**.

You need to look up an appropriate projection for the spatial extents of your data/map. This is a huge topic in and of itself, so for today we'll just focus on some (over-simplified) rules of thumb:

  1. If you're mapping global data, choose a global projection like [Pseudo-Mercator](https://epsg.io/3857) or [Robinson](https://epsg.io/53030).
  2. If you're mapping national data, choose a national projection like [LCC USA](https://epsg.io/42103).
  3. If you're mapping regional data, choose a local projection, like [UTM zone 11N](https://epsg.io/32611) for Southern California.
  
![](https://github.com/gboeing/ppd430/blob/fall/modules/07-spatial-data/img/utm_zones.png?raw=1)

https://epsg.io/ is a good resource. There you can use the EPSG code or click the "proj4" link on any CRS's page to get a string you can use with geopandas.

In [ ]:
# plot the tracts, then the listings on top of them
ax = gdf_tracts.plot()
ax = gdf_listings.plot(ax=ax, c="r", markersize=1)

In [ ]:
# define a CRS appropriate for projecting CA data
ca_crs = "epsg:26945"
gdf_tracts = gdf_tracts.to_crs(ca_crs)
gdf_listings = gdf_listings.to_crs(ca_crs)

In [ ]:
# plot the projected tracts + listings
ax = gdf_tracts.plot()
ax = gdf_listings.plot(ax=ax, c="r", markersize=1)

In [ ]:
# specify a projection manually with a proj4 string
# we'll map with UTM zone 11 which is good for Southern California (see link above)
utm_11 = "+proj=utm +zone=11 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
gdf_tracts = gdf_tracts.to_crs(utm_11)
gdf_listings = gdf_listings.to_crs(utm_11)

In [ ]:
ax = gdf_tracts.plot()
ax = gdf_listings.plot(ax=ax, c="r", markersize=1)

In [ ]:
# buffer listings by 5km then plot again
ax = gdf_tracts.plot()
ax = gdf_listings.buffer(5000).plot(ax=ax, fc="r")

So that's our projected data and shapefile. Notice how the shape has changed, and how the units make more sense: they are in meters now. So our buffers are a 5km radius from each point. Buffers are useful for tasks like, for example, finding all the transit stations within 1km of a census tract.

## 6. Non-spatial merge vs spatial joins

### 6a. Quick review of pandas (non-spatial) merge

Joins two dataframes based on some column they have in common.

In [ ]:
# load the CA tract-level census data from previous weeks
tract_indicators = pd.read_csv(
    "https://raw.githubusercontent.com/gboeing/ppd534/main/data/census_tracts_data_ca.csv",
    dtype={"GEOID10": str},
)
tract_indicators.shape

In [ ]:
# 5 rows of the tracts census data
tract_indicators.head()

In [ ]:
# 5 rows of the tracts shapefile
gdf_tracts.head()

In [ ]:
# merge the 2 datasets on a shared column: tract fips code
gdf_tracts_ind = pd.merge(
    left=gdf_tracts, right=tract_indicators, how="left", left_on="GEOID", right_on="GEOID10"
)
gdf_tracts_ind.head()

In [ ]:
# merging a dataframe (right) into a (left) geodataframe, we got a geodataframe back and kept our CRS
gdf_tracts_ind.crs

### 6b. Geopandas spatial join

Joins two geodataframes based on some shared spatial location. Let's say I want to know which county each rental listing is in: I need to *join* the listings to the county that each listing is within, using the gdf_counties GeoDataFrame we created earlier by dissolving the tracts.

In [ ]:
# remember (again): always double-check CRS before any spatial operations
gdf_listings.crs == gdf_counties.crs

In [ ]:
# they don't match, so project the counties to the CRS of the listings
gdf_counties = gdf_counties.to_crs(gdf_listings.crs)
gdf_listings.crs == gdf_counties.crs

In [ ]:
# spatial join listings to counties
# this is a left-join to ensure we retain all the listings in our resulting merged dataset
gdf_listings_counties = gpd.sjoin(gdf_listings, gdf_counties, how="left", predicate="within")
gdf_listings_counties.shape

In [ ]:
# what did it do? inspect first 5 rows of listings
gdf_listings_counties.head()

In [ ]:
groups = gdf_listings_counties.groupby("COUNTYFP")

In [ ]:
# which counties have the highest median asking rents?
groups["rent"].median().sort_values(ascending=False)

In [ ]:
# which counties have the most bedrooms per unit in the listings?
groups["bedrooms"].mean().sort_values(ascending=False)

In [ ]:
# which counties have the most listings?
groups["geometry"].count().sort_values(ascending=False)

In [ ]:
# create a subset of only those listings in orange county
# equivalent to gdf_listings.within(oc_geometry) from earlier
oc_listings = gdf_listings_counties[gdf_listings_counties["COUNTYFP"] == "059"]
oc_listings.shape

## 7. Mapping

In [ ]:
# get the merged tracts + indicators that are in OC
oc_tracts_ind = gdf_tracts_ind[gdf_tracts_ind["COUNTYFP"] == "059"]
oc_tracts_ind.head()

In [ ]:
# drop the tract that has 0 land area... it's just in the ocean
# census bureau uses these to represent territory boundary without making on-land tracts extend out into ocean
oc_tracts_ind = oc_tracts_ind[oc_tracts_ind["ALAND"] > 0]

In [ ]:
# map using GeoDataFrame plot method with some style configurations
ax = oc_tracts_ind.plot(
    column="med_household_income", cmap="plasma", edgecolor="k", lw=0.2, figsize=(9, 6), legend=True
)

# turn the "axis" off and save to disk
ax.axis("off")
ax.get_figure().savefig("oc-income.png", dpi=300, bbox_inches="tight")

In [ ]:
# map a different column
ax = oc_tracts_ind.plot(
    column="pct_english_only", cmap="plasma", edgecolor="k", lw=0.2, figsize=(9, 6), legend=True
)
ax.axis("off")
ax.get_figure().savefig("oc-language.png", dpi=300, bbox_inches="tight")

In [ ]:
# map tracts as a basemap with listings as points on top
ax = oc_tracts_ind.plot(facecolor="#aaaaaa", edgecolor="w", lw=0.5, figsize=(12, 9), legend=False)

# now plot listings, colored by asking rent
ax = oc_listings.dropna().plot(
    ax=ax, markersize=10, legend=True, cmap="plasma", column="rent", scheme="Quantiles"
)
ax.axis("off")
ax.set_title("Apartment Listings in Orange County by Asking Rent (USD)")
ax.get_figure().savefig("oc-listings.png", dpi=300, bbox_inches="tight")

In [ ]:
# now it's your turn
# make the plot above more effective and accessible using the visualization best practices you have read about